In [15]:
#IMPORTS
import requests as req
from bs4 import BeautifulSoup as bs
import re
from selenium import webdriver

In [16]:
class AutoPurchase:
    '''
    Checks to see if an item is in stock then sends an alert or purchases the item automatically.
    Do not use this class directly. Use the subclasses defined seperately for each store.
    '''
    def __init__(self,
                 alert=False,
                 purchase=False,
                 username=None,
                 password=None,
                 num_purchases=1):
        '''
        :param alert: the 10-digit phone number a text will be sent to if item is available, otherwise False
        :param purchase: True/False, if True must include the username and password parameters
        :param username: the username for the website where the item will be purchased from
        :param password: the password for the website where the item will be purchased from
        :param num_purchases: the number of items you wish to be purchased when available
        '''
        self.alert = alert
        self.purchase = purchase
        self.username = username
        self.password = password
        self.num_purchases = num_purchases
            
        if self.alert is False and self.purchase is False:
            raise Exception("Please either supply a phone number for an alert or the information required for a item "
                            + "purchase")
            
        if self.purchase is True and (self.username is None or self.password is None):
            raise Exception("when purchase is set to true you must supply a valid username and password")
    
    
    def __purchase_item(self):
        pass
    
    def __send_alert(self):
        pass
    
    def test(self):
        pass

In [17]:
class BestBuyAutoPurchase(AutoPurchase):
    '''
    Checks to see if an item is in stock at bestbuy.com then sends an alert or purchases the item automatically.
    '''
    _regex_site = '\Ahttps://www.bestbuy.com'
    
    def __init__(self,
                 alert=False,
                 purchase=False,
                 username=None,
                 password=None,
                 num_purchases=1):
        '''
        :param alert: the 10-digit phone number a text will be sent to if item is available, otherwise False
        :param purchase: True/False, if True must include the username and password parameters
        :param username: the username for the website where the item will be purchased from
        :param password: the password for the website where the item will be purchased from
        :param num_purchases: the number of items you wish to be purchased when available
        '''
        super().__init__(alert,purchase,username,password,num_purchases)
    
    def watch_single_item(self, url=None):
        '''
        :param url: the address the item is located at
        '''
        self.url = url
        site_check = re.match(self._regex_site, url)
        if not site_check:
            print(f'site check was {site_check.start}')
            raise Exception("The website is not https://bestbuy.com. Check the url you supplied and try again")
        
        checking = True
        num_purchased = 0
        while checking:
            if self.__check_inventory():
                if self.purchase:
                    try:
                        self.__purchase_item()
                    except Exception as err:
                        print(f"The item purchase failed: {err}\n")
                        print(sys.exc_info()[0])
                    else:
                        print("Item purchased")
                        num_purchased += 1
                        if num_purchased >= num_purchases:
                            checking = False
                else:
                    checking = False
                    
                if self.alert:
                    print(f"Item in stock. Sending alert to {self.alert}")
                    try:
                        self.__send_alert()
                    except Exception as err:
                        print(f'The alert failed to send: {err}')
                        print(sys.exc_info()[0])
                    else:
                        print('Alert sent')
        print('Script finished')
                        
                        
    def __get_html(self):
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'}
        page = req.get(self.url, headers=headers)
        if page.status_code != 200:
            raise Exception(f"The page returned a status code that was not 200. Status code was {page.status_code}")
        return page.content                    

    
    def __check_inventory(self):
        '''Returns True if item is in stock'''
        
        content = self.__get_html()
        soup = bs(content, 'html.parser')
        x = soup.find('button', class_="btn btn-primary btn-lg btn-block btn-leading-ficon add-to-cart-button")
        if x is not None:
            return True
        else:
            return False
        
    
    def __purchase_item(self):
        #need to configure it to purchase multiple items once I have it capable of buying 1 item
        pass
    
    def __send_alert(self):
        pass
    
    def test(self):
        pass

In [63]:
class NewEggAutoPurchase(AutoPurchase):
    '''
    Checks if items from newegg.com are available for purchase. If so can send alerts or purchase the item.
    Can be set to watch a single item or to watch every item returned from a search of in which the 
    search_term appears in the item description.
    '''
    _regex_site = '\Ahttps://www.newegg.com'
    
    def __init__(self,
                 alert=False,
                 purchase=False,
                 username=None,
                 password=None,
                 num_purchases=1,):
        '''
        :param alert: the 10-digit phone number a text will be sent to if item is available, otherwise False
        :param purchase: True/False, if True must include the username and password parameters
        :param username: the username for the website where the item will be purchased from
        :param password: the password for the website where the item will be purchased from
        :param num_purchases: the number of items you wish to be purchased when available
        '''
        super().__init__(alert,purchase,username,password,num_purchases)
        
    
    def watch_single_item(self, url=None):
        '''
        :param url: the address the item is located at
        '''
        self.url = url
        site_check = re.match(self._regex_site, url)
        if not site_check:
            print(f'site check was {site_check.start}')
            raise Exception("The website is not https://newegg.com. Check the url you supplied and try again")
        
        checking = True
        num_purchased = 0
        while checking:
            if self.__check_inventory_single():
                if self.purchase:
                    try:
                        self.__purchase_item()
                    except Exception as err:
                        print(f"The item purchase failed: {err}\n")
                        print(sys.exc_info()[0])
                    else:
                        print("Item purchased")
                        num_purchased += 1
                        if num_purchased >= num_purchases:
                            checking = False
                else:
                    checking = False
                    
                if self.alert:
                    print(f"Item in stock. Sending alert to {self.alert}")
                    try:
                        self.__send_alert()
                    except Exception as err:
                        print(f'The alert failed to send: {err}')
                        print(sys.exc_info()[0])
                    else:
                        print('Alert sent')
        print('Script finished')
        
    def watch_multiple_items(self, search_term=None):
        '''
        Searches on new egg for the search_term then monitors the stock of items on the first
        results page that have descriptions matching the search term. Alerts will be sent for each
        item as it becomes available. Items will be purchased as they become available until the number
        of items purchases is equal to self.num_purchases
        '''
        
        if search_term is None:
            raise Exception('You must supply a string for the search term')
            
        term_fix = search_term.strip().lower().replace(' ', '+')
        self.url = f'https://www.newegg.com/p/pl?d={term_fix}'
        
        content = self.__get_html()
        soup = bs(content, 'html.parser')
        item_listings = soup.find_all(class_="item-cell")
        
        items_to_watch = []
        for x in item_listings:
            listing = x.find(class_='item-img')
            title = listing.img['title'].lower()
            if search_term in title:
                link_to_product = listing['href'].replace(' ', '%20')
                element_id = x['id']
                xpath = f'//*[@id="{element_id}"]/div/div[2]/div/div[1]/button'
                items_to_watch.append({'title':title, 'link_to_product':link_to_product, 'element_id':element_id, 'xpath':xpath})
        
        while True:
            self.__check_inventory_multiple(items_to_watch)
                
                
    def __get_html(self):
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'}
        page = req.get(self.url, headers=headers)
        if page.status_code != 200:
            raise Exception(f"The page returned a status code that was not 200. Status code was {page.status_code}")
        return page.content
    
    def __check_inventory_single(self):
        '''Returns True if item is in stock'''
        
        content = self.__get_html()
        soup = bs(content, 'html.parser')
        # add the logic for figuring out if an item is in stock on its specifc item page
        if x is not None:
            return True
        else:
            return False
        
    def __check_inventory_multiple(self, items_to_watch={}):
        '''Returns True if item is in stock'''
        if not items_to_watch:
            raise Exception('No item descriptions matched search query')
        content = self.__get_html()
        soup = bs(content, 'html.parser')
        for item in items_to_watch:
            ele_id = item['element_id']
            item_cell = soup.find(id=ele_id)
            in_stock = False if item_cell.find(class_='btn btn-primary btn-mini') is None else True
            if in_stock:
                if self.alert:
                    self.__send_alert(ele_id)
                    items_to_watch.pop('element_id'==ele_id)
            else:
                print('not in stock')
    def __purchase_item(self):
        pass
    
    def __send_alert(self, id):
        print(f'sending alert for {id}')
        
    
    def test(self):
        pass

In [19]:
test = BestBuyAutoPurchase(alert=2254564615)
test.watch_single_item(url='https://www.bestbuy.com/site/toshiba-32-class-led-hd-smart-firetv-edition-tv/6398132.p?skuId=6398132')

Item in stock. Sending alert to 2254564615
Alert sent
Script finished


In [64]:
test = NewEggAutoPurchase(alert=2254564615)
test.watch_multiple_items('radeon rx 570 directx 12')

not in stock
not in stock
sending alert for item_cell_14-202-384_3_0
not in stock
sending alert for item_cell_9SIA73MBC31134_6_0
not in stock
sending alert for item_cell_9SIAE8D7F32388_9_0
not in stock
sending alert for item_cell_9SIAHT8BKN1374_12_0
sending alert for item_cell_9SIA73M8NS6506_14_0
sending alert for item_cell_9SIA73M86W8794_16_0
not in stock
sending alert for item_cell_9SIAMY5BGU5391_20_0
not in stock
not in stock
not in stock
not in stock
not in stock
not in stock
sending alert for item_cell_9SIAE8D7F32388_9_0
not in stock
sending alert for item_cell_9SIAHT8BKN1374_12_0
sending alert for item_cell_9SIA73M8NS6506_14_0
sending alert for item_cell_9SIA73M86W8794_16_0
not in stock
sending alert for item_cell_9SIAMY5BGU5391_20_0
not in stock
not in stock
not in stock
not in stock
not in stock
sending alert for item_cell_9SIA4RE91C0928_13_0
sending alert for item_cell_9SIA8UCBUD9457_15_0
sending alert for item_cell_9SIAMY5B977602_17_0
sending alert for item_cell_9SIAMY5BGU539

KeyboardInterrupt: 

In [9]:
url_in = 'https://www.bestbuy.com/site/toshiba-32-class-led-hd-smart-firetv-edition-tv/6398132.p?skuId=6398132'
url_out = 'https://www.bestbuy.com/site/nvidia-geforce-rtx-3060-ti-8gb-gddr6-pci-express-4-0-graphics-card-steel-and-black/6439402.p?skuId=6439402'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'}
page = req.get(url_in, headers=headers)
if page.status_code != 200:
    raise Exception(f"The page returned a status code that was not 200. Status code was {page.status_code}")
content = page.content
soup = bs(content, 'html.parser')
x=soup.find('button', class_="btn btn-primary btn-lg btn-block btn-leading-ficon add-to-cart-button")
if x is not None:
    print('True')
else:
    print('False')

True


In [10]:
_regex_site = '\Ahttps://www.bestbuy.com'
url = 'https://www.bestby.com/site/nvidia-geforce-rtx-3060-ti-8gb-gddr6-pci-express-4-0-graphics-card-steel-and-black/6439402.p?skuId=6439402'
site_check = re.match(_regex_site, url)
print(f'site check was {site_check}')
if not site_check: #and not override1:
    print(f'site check was {site_check}')
    raise Exception("The website is not https://bestbuy.com. If you are SURE this error is wrong, pass the "
        + "parameter override1=True into the object initialization code to force creation")
    

site check was None
site check was None


Exception: The website is not https://bestbuy.com. If you are SURE this error is wrong, pass the parameter override1=True into the object initialization code to force creation

In [11]:
rtx 3060 ti
rtx 3070

SyntaxError: invalid syntax (<ipython-input-11-68886f55ce8c>, line 1)

# add to cart button
<button class="btn btn-primary btn-mini" title="Add PowerColor RED DRAGON Radeon RX 570 DirectX 12 AXRX 570 4GBD5-DHDV3/OC 4GB 256-Bit GDDR5 PCI Express 3.0 CrossFireX Support ATX Video Card to cart">Add to cart <i class="fas fa-caret-right"></i></button>
# auto notify button
<button class="btn btn-secondary btn-mini" title="Auto Notify ">Auto Notify <i class="fas fa-caret-right"></i></button>
# sold out button
<span class="btn btn-message btn-mini">Sold Out</span>


# item listing on search page
<div class="item-cell" id="item_cell_14-131-766__0">
    <div class="item-container">
        <a href="https://www.newegg.com/powercolor-radeon-rx-570-axrx-570-4gbd5-dhdv3-oc/p/N82E16814131766" class="item-img">
            <img src="https://c1.neweggimages.com/ProductImageCompressAll300/14-131-766-S02.jpg" title="PowerColor RED DRAGON Radeon RX 570 DirectX 12 AXRX 570 4GBD5-DHDV3/OC 4GB 256-Bit GDDR5 PCI Express 3.0 CrossFireX Support ATX Video Card" alt="PowerColor RED DRAGON Radeon RX 570 DirectX 12 AXRX 570 4GBD5-DHDV3/OC 4GB 256-Bit GDDR5 PCI Express 3.0 CrossFireX Support ATX Video Card" class="checkedimg">
        </a>
        <div class="item-info">
            <div class="item-branding">
                <a href="https://www.newegg.com/PowerColor/BrandStore/ID-1419" class="item-brand">
                    <img src="https://c1.neweggimages.com/Brandimage_70x28/Brand1419.gif" title="PowerColor" alt="PowerColor">
                </a>
            </div>
            <a href="https://www.newegg.com/powercolor-radeon-rx-570-axrx-570-4gbd5-dhdv3-oc/p/N82E16814131766" class="item-title" title="View Details">PowerColor RED DRAGON Radeon RX 570 DirectX 12 AXRX 570 4GBD5-DHDV3/OC 4GB 256-Bit GDDR5 PCI Express 3.0 CrossFireX Support ATX Video Card</a>
            <ul class="item-features">
                <li>
                    <strong>Max Resolution:</strong> DVI: 2560x1600
DisplayPort: 4096x2160
HDMI: 4096x2160</li><li><strong>DisplayPort:</strong> 1 x DisplayPort</li><li><strong>DVI:</strong> 1 x DL-DVI-D</li><li><strong>HDMI:</strong> 1 x HDMI</li><li><strong>Model #: </strong>AXRX 570 4GBD5-DHDV3</li><li><strong>Item #: </strong>N82E16814131766</li><li><strong>Return Policy: </strong><a href="https://kb.newegg.com/Article/Index/12/3?id=1167#53" target="_blank" title="Extended Holiday Return Policy(New Window)">Extended Holiday Return Policy</a></li></ul></div><div class="item-action"><ul class="price"><li class="price-was"></li><li class="price-map">&nbsp;</li><li class="price-current "><span class="price-current-label"></span>$<strong>169</strong><sup>.99</sup>&nbsp;<span class="price-current-range"><abbr title="to">–</abbr></span></li><li class="price-save "></li><li class="price-note"></li><li class="price-ship">$3.99 Shipping</li></ul><div class="item-operate"><div class="item-button-area"><button class="btn btn-primary btn-mini" title="Add PowerColor RED DRAGON Radeon RX 570 DirectX 12 AXRX 570 4GBD5-DHDV3/OC 4GB 256-Bit GDDR5 PCI Express 3.0 CrossFireX Support ATX Video Card to cart">Add to cart <i class="fas fa-caret-right"></i></button></div><div class="item-compare-box"><label class="form-checkbox"><input type="checkbox" autocomplete="off"><span class="form-checkbox-title">Compare</span></label></div></div></div><div class="item-stock" id="stock_14-131-766"></div></div></div>


In [12]:
url_egg = 'https://www.newegg.com/Desktop-Graphics-Cards/SubCategory/ID-48?Tid=7709'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'}

search_term = 'radeon rx 570 directx 12'
term_fix = search_term.strip().lower().replace(' ', '+')
search_url = f'https://www.newegg.com/p/pl?d={term_fix}'



page = req.get(search_url, headers=headers)
if page.status_code != 200:
    raise Exception(f"The page returned a status code that was not 200. Status code was {page.status_code}")
content = page.content
soup = bs(content, 'html.parser')
item_listings = soup.find_all(class_="item-cell")

items_to_watch = []
for x in item_listings:
    listing = x.find(class_='item-img')
    title = listing.img['title'].lower()
    if search_term in title:
        link_to_product = listing['href'].replace(' ', '%20')
        element_id = x['id']
        xpath = f'//*[@id="{element_id}"]/div/div[2]/div/div[1]/button'
        items_to_watch.append({'title':title, 'link_to_product':link_to_product, 'element_id':element_id, 'xpath':xpath})
                
    print(items_to_watch)


        

[{'title': 'asus rog strix radeon rx 570 directx 12 rog-strix-rx570-o8g-gaming 8gb 256-bit gddr5 pci express 3.0 hdcp ready video card', 'link_to_product': 'https://www.newegg.com/asus-radeon-rx-570-rog-strix-rx570-o8g-gaming/p/N82E16814126427?Description=radeon%20rx%20570%20directx%2012&cm_re=radeon_rx%20570%20directx%2012-_-14-126-427-_-Product', 'element_id': 'item_cell_14-126-427_1_0', 'xpath': '//*[@id="item_cell_14-126-427_1_0"]/div/div[2]/div/div[1]/button'}]
[{'title': 'asus rog strix radeon rx 570 directx 12 rog-strix-rx570-o8g-gaming 8gb 256-bit gddr5 pci express 3.0 hdcp ready video card', 'link_to_product': 'https://www.newegg.com/asus-radeon-rx-570-rog-strix-rx570-o8g-gaming/p/N82E16814126427?Description=radeon%20rx%20570%20directx%2012&cm_re=radeon_rx%20570%20directx%2012-_-14-126-427-_-Product', 'element_id': 'item_cell_14-126-427_1_0', 'xpath': '//*[@id="item_cell_14-126-427_1_0"]/div/div[2]/div/div[1]/button'}, {'title': 'gigabyte radeon rx 570 directx 12 gv-rx570gamin

In [ ]:
<button class="btn btn-primary btn-mini" title="Add GIGABYTE Radeon RX 570 DirectX 12 GV-RX570GAMING-4GD REV2.0 4GB 256-Bit GDDR5 PCI Express 3.0 x16 ATX Video Card to cart">Add to cart <i class="fas fa-caret-right"></i></button>

In [16]:
search_term = 'radeon rx 570 directx 12'
search_term = search_term.replace(' ', '+')
print(search_term)

radeon+rx+570+directx+12


In [11]:
#box that shows up once you click add to cart on the search page
<div class="modal right fade modal-intermediary show animation" role="dialog" id="modal-intermediary" style="display: none;"><div class="modal-dialog modal-lg" role="document"><div class="modal-content"><div class="modal-header"><div class="modal-title"><div class="message message-success no-bg"><div class="message-wrapper"><div class="message-icon"></div><div class="message-information"><div class="message-title">Item has been added to cart.</div></div></div></div></div><button type="button" class="close" data-dismiss="modal" aria-label="Close"><i class="fas fa-times"></i></button></div><div class="modal-body auto-height"><div class="item-actions"><div class="item-summary">Cart Subtotal (<!-- -->1  Item<!-- -->)<strong>:  $169.99</strong></div><button class="btn" title="Continue Shopping">Continue Shopping</button><button class="btn btn-undefined btn-primary" title="View Cart &amp; Checkout">View Cart &amp; Checkout</button></div></div></div></div></div>

SyntaxError: invalid syntax (<ipython-input-11-04497885a402>, line 2)

In [20]:
# add to cart button on search page
<button class="btn btn-primary btn-mini" title="Add GIGABYTE Radeon RX 570 DirectX 12 GV-RX570GAMING-4GD REV2.0 4GB 256-Bit GDDR5 PCI Express 3.0 x16 ATX Video Card to cart">Add to cart <i class="fas fa-caret-right"></i></button>

SyntaxError: invalid syntax (<ipython-input-20-f9270fc398cc>, line 2)

In [ ]:
document.querySelector("#item_cell_9SIAE8D7F32388_12_0 > div > div.item-action > div > div.item-button-area > button")
//*[@id="item_cell_9SIAE8D7935816_9_0"]/div/div[2]/div/div[1]/button
//*[@id="item_cell_9SIA8UCBUD9457_15_0"]/div/div[2]/div/div[1]/button